In [1]:
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import VectorStoreIndex
from llama_index.core.response.pprint_utils import pprint_source_node
import weaviate
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

client: weaviate.WeaviateClient = weaviate.connect_to_local()
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)
Settings.llm = Ollama(model="llama3:8b", request_timeout=60.0)

In [2]:
vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name="Pymupdf", text_key="content"
)

In [15]:
retriever = VectorStoreIndex.from_vector_store(vector_store).as_retriever(
    similarity_top_k=5
)

In [16]:
from typing import List
from llama_index.core.schema import NodeWithScore

# query: str = input("Please type your query: ")
query = "what was the maek and model of the EV sophie was interested in purchasing?"
query = "i am interested in purchasing a new car. tell me a little about plug in hybrids"
query = "what are PHEVs?"
query = "How many tpyes of motors do Plug-in hybrids (or PHEVs) have?"
nodes: List[NodeWithScore] = retriever.retrieve(query)

In [17]:
from langchain_core.prompt_values import PromptValue
from llama_index.core.base.llms.types import ChatMessage

context: str = ""
for n in nodes:
    context = context + f"""
    --------------------
    Exerpt from file with name: {n.node.metadata["properties"]["file_name"]}

    {n.text}
    --------------------
    """
prompt: list[ChatMessage] = [
    ChatMessage(role="system", content="You are an AI assistant that strictly relies on the provided context to answer user queries. Use only the information given in the context section to generate a response. If the context does not contain relevant information, inform the user accordingly. Do not use any prior knowledge."),
    ChatMessage(role="user", content="""Using the following context, generate a response to the query.
### Query:
{query}

### Context:
{context}""".format(query=query, context=context)),
]

In [18]:
response = Settings.llm.chat(prompt)

In [19]:
print(response)
print(context)

assistant: Based on the provided context, there is no mention of Plug-in hybrids (or PHEVs) having a specific number of types. The context only mentions Nissan Leaf and other electric vehicles or hybrid cars, but does not provide information about PHEVs. Therefore, I cannot generate an answer to your query.

If you meant to ask something else related to the provided context, please feel free to rephrase your question!

    --------------------
    Exerpt from file with name: Is it worth buying a second-hand Nissan Leaf.txt

    s 12 bars._

There’s an app you can get called Leaf Spy which connects to an EV using a dongle that gives you a report on
the battery health of the EV. Seeing the Leaf Spy report gave us the confidence that the Leaf we were looking to
buy had the battery life that was advertised.

#### How does having two cars, an EV and a petrol car, work for your family?

I use the EV all the time. I work in Lower Hutt, we live in Upper Hutt (Wellington) and the kids go to sch

In [20]:
client.close()